In [1]:
!pip install -q kaggle

# Create a directory and move the kaggle.json file to it to ensure the correct permissions
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [2]:
!kaggle datasets download -d dheerajperumandla/drowsiness-dataset


 92% 148M/161M [00:01<00:00, 106MB/s]
100% 161M/161M [00:01<00:00, 109MB/s]


In [3]:
!unzip -q drowsiness-dataset.zip -d drowsiness_dataset


In [5]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam


In [6]:
# Define input image size
input_shape = (224, 224, 3)

# Define batch size
batch_size = 32

# Create data generators for training and validation
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    '/content/drowsiness_dataset/train',
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    '/content/drowsiness_dataset/train',
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)


Found 2321 images belonging to 4 classes.
Found 579 images belonging to 4 classes.


In [7]:
# Load pre-trained GoogLeNet model
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)

# Add custom layers on top of the pre-trained model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
output = Dense(4, activation='softmax')(x)  # Assuming 4 classes: Open, Closed, Yawn, NoYawn

# Create the new model
model = Model(inputs=base_model.input, outputs=output)


87910968/87910968 [==============================] - 1s 0us/step


In [12]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
epochs = 10
history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples // batch_size,
                    validation_data=validation_generator,
                    validation_steps=validation_generator.samples // batch_size,
                    epochs=epochs)


Epoch 1/10
13/72 [====>.........................] - ETA: 18:19 - loss: 0.3588 - accuracy: 0.8341

KeyboardInterrupt: ignored